### **Método 2: Cálculo mediante diferencias finitas**

In [ ]:
# Importación de librerías
from ipywidgets import interactive
import ipywidgets as widgets
from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.figure import Figure

In [ ]:
def CalculoPosicionInicial(matrizPosicionInicial, puntosMalla, x, L):
    """
    Se realiza el cálculo inicial de la posición con t=0 s.

    Parámetros de la función:
    ------------------------
    x: Posición en el eje X.
    L: Constante definida para el problema.
    
    Salida de la función
    ---------------------
    Matriz de la posición inicial en x y t
    """
    for i in range (0, puntosMalla):
      if x[i] <= 4*L/5:
        matrizPosicionInicial [i][0] = 5*x[i]/(4*L)
      elif x[i] > 4*L/5:
        matrizPosicionInicial [i][0] = 5 - 5*x[i]/L
    else:
      return matrizPosicionInicial

In [ ]:
def CalculoEcuacion(matrizEcuacion, precision, puntosMalla, puntosTiempo, L, p, T):
    """
    Parámetros de la funcion:
    ------------------------
    matrizEcuacion: Matriz donde se almacenan los valores de tiempo y espacio
    precision: Precision deseada que define la cantidad de iteraciones del metodo.
    puntosMalla: Puntos en los que se va a evaluar la ecuacion.
    puntosTiempo: Puntos de tiempo en los que se va a evaluar la ecuacion.
    L: Longitud
    p: Densidad del medio
    T: Tensión
    
    Salida de la funcion
    ---------------------
    Matriz con la solución de la ecuación de onda en el tiempo y posición.
    """

    # Se define el inicio del contador de iteraciones 
    contadorIteraciones = 0
    # Se define un contador que lleve la cantidad de veces que no se cumple la presición deseada.
    cantidadImprecisiones = 1
    
    #Se define un ciclo que funcione hasta que deje de cumplirse la presición.
    while cantidadImprecisiones > 0:
        contadorIteraciones += 1
        cantidadImprecisiones = 0
    
        for contadorPosición in range (0, puntosMalla - 2):
            for contadorTiempo in range (1, puntosMalla - 1):

                EcuacionAnterior = matrizEcuacion[contadorPosición+1][contadorTiempo]
                gamma = p*(tamañoLado/puntosMalla)**2/(T*(puntosTiempo/puntosMalla)**2)
 
                matrizEcuacion[contadorPosición+1][contadorTiempo] = gamma*(matrizEcuacion[contadorPosición+1][contadorTiempo]
                -2*matrizEcuacion[contadorPosición][contadorTiempo]+matrizEcuacion[contadorPosición-1][contadorTiempo])
                +2*matrizEcuacion[contadorPosición][contadorTiempo]-matrizEcuacion[contadorPosición-1][contadorTiempo]

                #Se calcula la diferencia entre la difusión anterior y la actual
                diferenciaEcuacion = np.abs(EcuacionAnterior - matrizEcuacion[contadorPosición+1][contadorTiempo])     

                #Se plantea la condición de parada con la presición deseada y la diferencia de difusión           
                if diferenciaEcuacion > precisión:
                    cantidadImprecisiones += 1
                if contadorIteraciones > 500:
                    cantidadImprecisiones = 0
               
    return matrizEcuacion, contadorIteraciones

In [ ]:
def Grafico_interactivo(L, p, Ten):
  '''Resuelve la ecuación de onda para una cuerda de densidad lineal de masa
  lambda, de longitud L, sometida a una tensión constante T, cuyo desplazamiento
  ocurre en la dirección y.
 
  Parámetros de la función
  ------------------------
  L: longitud
  p: densidad del medio
  T: tensión
 
  Salida de la función
  --------------------
  y(x,t) : conjunto de valores de la solución de la ecuación de onda.
  '''
 
  # Se definen las variables para resolver el problema
  tamañoLado = 10
  puntosTiempo = 30

  # Se definen los puntos en los que se va a evaluar la ecuación
  puntosMalla = 50
  t = np.linspace (0, puntosTiempo, puntosMalla)
  x = np.linspace (0, L, puntosMalla)
  T, X = np.meshgrid (t,x)

  # Se crea una matriz de ceros del tamaño antes definido
  matrizPosicionInicial = np.zeros ((puntosMalla, puntosMalla))

  # Se modifican los valores para el tiempo inicial
  matrizPosicionInicial = CalculoPosicionInicial(matrizPosicionInicial, puntosMalla, x, L)

  # Se realiza el cálculo de la ecuación
  resultadoFinal = CalculoEcuacion(matrizPosicionInicial, precision, puntosMalla, puntosTiempo, L, p, Ten)
    
  # Se definen las variables que contienen los resultados
  EcuacionFinal =  resultadoFinal [0]
  iteracionesTotales = resultadoFinal [1]
    
  # Se grafican los resultados
  fig = plt.figure (figsize=(8, 6), dpi=80)
  ax = plt.axes (projection = '3d') 
  ax.set_title (r"Ecuación de onda para una cuerda dependiente del"
  "\n" 
  r"tiempo y la posición por método de Diferencias Finitas")   
  ax.set_ylabel ('t (s)')
  ax.set_xlabel ('x (m)')
  ax.set_zlabel ('y (t,x) (m)')
  ax.plot_surface (X, T, EcuacionFinal  , rstride=1, cstride=1, cmap= 'cividis')
 
  return

In [ ]:
y=interactive(Grafico_interactivo, {'manual': True}, L=widgets.IntSlider(min=1.0, max=15.0, step=1.0, value=10.0, description='Longitud :'),
              p=widgets.IntSlider(min=1, max=10, step=1.0, value=5.0, description='Densidad :'),
              Ten=widgets.IntSlider(min=15, max=50, step=5.0, value=25.0, description='Tensión :'))
display(y)

interactive(children=(IntSlider(value=10, description='Longitud :', max=15, min=1), IntSlider(value=5, descrip…